# 目的

[Deep Anomaly Detection with Deviation Networks](https://arxiv.org/abs/1911.08623)のPytorch実装

# 環境の準備

In [ ]:
!pip install torchsummaryX

In [ ]:
# data hundling
import pandas as pd
import numpy as np
from scipy.sparse import vstack, csc_matrix

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc,roc_curve, precision_recall_curve, average_precision_score, roc_auc_score
# pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.optim import RMSprop
from torchsummaryX import summary
# 
from tqdm.notebook import tqdm

# モデルの定義

In [ ]:
class DeepDevNet(nn.Module):
  def __init__(self, input_num):
    super(DeepDevNet, self).__init__()
    self.input_num = input_num

    self.linear_0 = nn.Linear(input_num, 1000)
    self.linear_1 = nn.Linear(1000, 250)
    self.linear_2 = nn.Linear(250, 20)
    self.linear_3 = nn.Linear(20, 1)

    self.relu = nn.ReLU()
  
  def forward(self, x):
    x = self.relu(self.linear_0(x))
    x = self.relu(self.linear_1(x))
    x = self.relu(self.linear_2(x))
    x = self.linear_3(x)
    return x

In [ ]:
class ShortDevNet(nn.Module):
  def __init__(self, input_num):
    super(ShortDevNet, self).__init__()
    self.input_num = input_num

    self.linear_0 = nn.Linear(input_num, 20)
    self.linear_1 = nn.Linear(20, 1)

    self.relu = nn.ReLU()
  
  def forward(self, x):
    x = self.relu(self.linear_0(x))
    x = self.linear_1(x)
    return x

In [ ]:
class LinearDevNet(nn.Module):
  def __init__(self, input_num):
    super(LinearDevNet, self).__init__()
    self.input_num = input_num

    self.linear_0 = nn.Linear(input_num, 1)
  
  def forward(self, x):
    x = self.linear_0(x)
    return x

In [ ]:
class DevNet(nn.Module):
  def __init__(self, input_num, network_depth):
    super(DevNet, self).__init__()
    self.input_num = input_num
    self.network_depth = network_depth

    if network_depth == 4:
      self.devnet = DeepDevNet(input_num)
    elif network_depth == 2:
      self.devnet = ShortDevNet(input_num)
    elif network_depth == 1:
      self.devnet = LinearDevNet(input_num)
    else:
      print("Can't Defined")
  def forward(self, x):
    x = self.devnet(x)
    return x

# DataSet定義

In [ ]:
class DevDataset(Dataset):
  def __init__(self, x, y=None, mode="train"):
    self.x = x
    self.y = y
    self.mode = mode
  
  def __len__(self):
    return len(self.x)
  
  def __getitem__(self, index):
    item_x = self.x[index]
    if self.mode == "train":
      item_y = self.y[index]

      return {
          "x": torch.tensor(item_x, dtype=torch.float32),
          "y": torch.tensor(item_y, dtype=torch.float32)
      }
    else:
      return {
          "x": torch.tensor(item_x, dtype=torch.float32),
      }

# 損失関数の定義

In [ ]:
class DeviationLoss(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, y_pred, y_true):
        confidence_margin = 5.
        # size=5000 is the setting of l in algorithm 1 in the paper
        ref = torch.normal(mean=0., std=torch.full([5000], 1.)).cuda()
        dev = (y_pred - torch.mean(ref)) / torch.std(ref)
        inlier_loss = torch.abs(dev)
        outlier_loss = torch.abs((confidence_margin - dev).clamp_(min=0.))
        return torch.mean((1 - y_true) * inlier_loss + y_true * outlier_loss)

In [ ]:
def aucPerformance(mse, labels):
    roc_auc = roc_auc_score(labels, mse)
    ap = average_precision_score(labels, mse)
    print("AUC-ROC: %.4f, AUC-PR: %.4f" % (roc_auc, ap))
    return roc_auc, ap;

# 前処理関数定義

In [ ]:
def data_generator_sup(x, outlier_indices, inlier_indices, batch_size, nb_batch, rng):
    """batch generator
    """
    rng = np.random.RandomState(rng.randint(MAX_INT, size = 1))
    counter = 0             
    ref, training_labels = input_data_generation_sup(x, outlier_indices, inlier_indices, batch_size*nb_batch, rng)
    return ref, training_labels

In [ ]:
def input_data_generation_sup(x_train, outlier_indices, inlier_indices, batch_size, rng):
    '''
    batchs of samples. This is for csv data.
    Alternates between positive and negative pairs.
    '''      
    dim = x_train.shape[1]
    ref = np.empty((batch_size, dim))    
    training_labels = []
    n_inliers = len(inlier_indices)
    n_outliers = len(outlier_indices)
    for i in range(batch_size):    
        if(i % 2 == 0):
            sid = rng.choice(n_inliers, 1)
            ref[i] = x_train[inlier_indices[sid]]
            training_labels += [0]
        else:
            sid = rng.choice(n_outliers, 1)
            ref[i] = x_train[outlier_indices[sid]]
            training_labels += [1]
    return np.array(ref), np.array(training_labels)

In [ ]:
def input_batch_generation_sup_sparse(x_train, outlier_indices, inlier_indices, batch_size, rng):
    '''
    batchs of samples. This is for libsvm stored sparse data.
    Alternates between positive and negative pairs.
    '''      
    ref = np.empty((batch_size))    
    training_labels = []
    n_inliers = len(inlier_indices)
    n_outliers = len(outlier_indices)
    for i in range(batch_size):    
        if(i % 2 == 0):
            sid = rng.choice(n_inliers, 1)
            ref[i] = inlier_indices[sid]
            training_labels += [0]
        else:
            sid = rng.choice(n_outliers, 1)
            ref[i] = outlier_indices[sid]
            training_labels += [1]
    ref = x_train[ref, :].toarray()
    return ref, np.array(training_labels)

In [ ]:
def inject_noise(seed, n_out, random_seed):   
    '''
    add anomalies to training data to replicate anomaly contaminated data sets.
    we randomly swape 5% features of anomalies to avoid duplicate contaminated anomalies.
    this is for dense data
    '''  
    rng = np.random.RandomState(random_seed) 
    n_sample, dim = seed.shape
    swap_ratio = 0.05
    n_swap_feat = int(swap_ratio * dim)
    noise = np.empty((n_out, dim))
    for i in np.arange(n_out):
        outlier_idx = rng.choice(n_sample, 2, replace = False)
        o1 = seed[outlier_idx[0]]
        o2 = seed[outlier_idx[1]]
        swap_feats = rng.choice(dim, n_swap_feat, replace = False)
        noise[i] = o1.copy()
        noise[i, swap_feats] = o2[swap_feats]
    return noise

# 実行

## 検証データ読み込み

In [ ]:
!git clone https://github.com/GuansongPang/deviation-network.git

fatal: destination path 'deviation-network' already exists and is not an empty directory.


## パラメータ設定

In [ ]:
DATA_PATH = "/content/deviation-network/dataset/annthyroid_21feat_normalised.csv"

NAME = "annthyroid"
SEED = 0
MAX_INT = np.iinfo(np.int32).max
NETWORK_DEPTH = 2
RUNS = 10
CONT_RATE = 0.02
KNOWN_OUTLIERS = 30
EPOCHS = 50
BATCH_SIZE = 512

## 初期化

In [ ]:
rauc = np.zeros(RUNS)
ap = np.zeros(RUNS) 

train_time = 0
test_time = 0

## データロード

In [ ]:
train_df = pd.read_csv(DATA_PATH)
train_df.head()

,Dim_0,Dim_1=0,Dim_2=0,Dim_3=0,Dim_4=0,Dim_5=0,Dim_6=0,Dim_7=0,Dim_8=0,Dim_9=0,Dim_10=0,Dim_11=0,Dim_12=0,Dim_13=0,Dim_14=0,Dim_15=0,Dim_16,Dim_17,Dim_18,Dim_19,Dim_20,class
0,0.750000,1,0,1,1,1,1,1,0,1,1,1,1,1,1,1,0.001132,0.080780,0.197324,0.300926,0.225000,0
1,0.239583,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.000472,0.164345,0.235786,0.537037,0.165625,0
2,0.479167,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.003585,0.130919,0.167224,0.527778,0.118750,0
3,0.656250,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.001698,0.091922,0.125418,0.337963,0.129688,0
4,0.229167,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.000472,0.142061,0.229097,0.337963,0.235938,0


In [ ]:
x = train_df.drop("class", axis=1).values
labels = train_df["class"]

In [ ]:
outlier_indices = np.where(labels == 1)[0]
outliers = x[outlier_indices]  
n_outliers_org = outliers.shape[0]  

## TRAINING

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, labels, test_size=0.2, random_state=42, stratify = labels)

In [ ]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
outlier_indices = np.where(y_train == 1)[0]
inlier_indices = np.where(y_train == 0)[0]
n_outliers = len(outlier_indices)

In [ ]:
n_noise  = len(np.where(y_train == 0)[0]) * CONT_RATE / (1. - CONT_RATE)
n_noise = int(n_noise) 

In [ ]:
rng = np.random.RandomState(SEED)

In [ ]:
if n_outliers > KNOWN_OUTLIERS:
    mn = n_outliers - KNOWN_OUTLIERS
    remove_idx = rng.choice(outlier_indices, mn, replace=False)            
    x_train = np.delete(x_train, remove_idx, axis=0)
    y_train = np.delete(y_train, remove_idx, axis=0)

noises = inject_noise(outliers, n_noise, SEED)
x_train = np.append(x_train, noises, axis = 0)
y_train = np.append(y_train, np.zeros((noises.shape[0], 1)))

In [ ]:
outlier_indices = np.where(y_train == 1)[0]
inlier_indices = np.where(y_train == 0)[0]
print(y_train.shape[0], outlier_indices.shape[0], inlier_indices.shape[0], n_noise)
input_shape = x_train.shape[1:]
n_samples_trn = x_train.shape[0]
n_outliers = len(outlier_indices)            
print("Training data size: %d, No. outliers: %d" % (x_train.shape[0], n_outliers))

5471 30 5441 108
Training data size: 5471, No. outliers: 30


In [ ]:
input_shape = x_train.shape[1:][0]
epochs = EPOCHS
batch_size = BATCH_SIZE
nb_batch = 20

In [ ]:
model = DevNet(input_shape, NETWORK_DEPTH)

In [ ]:
gen_x, gen_y = data_generator_sup(x_train, outlier_indices, inlier_indices, batch_size, nb_batch, rng)

In [ ]:
dev_dataset = DevDataset(gen_x, gen_y, mode="train")
dev_dataloader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

In [ ]:
score.reshape(1,-1)[0]

tensor([ 0.0538,  0.0913,  0.1084,  0.1077,  0.1385,  0.0913,  0.0994,  0.1084,
         0.0677,  0.1060,  0.0913,  0.1063,  0.0507,  0.0897,  0.0847,  0.1056,
         0.1060,  0.0886,  0.1047,  0.0527,  0.0688,  0.1056,  0.1023,  0.0470,
         0.1186,  0.0913,  0.1059,  0.0886,  0.1110,  0.0467,  0.1074,  0.0470,
         0.0942,  0.0913,  0.1109,  0.1124,  0.0524,  0.1060,  0.0589,  0.0467,
         0.0583,  0.1077,  0.0755,  0.1094,  0.1116,  0.0935,  0.1092,  0.0527,
         0.1084,  0.1060,  0.0620,  0.0280,  0.1354,  0.1084,  0.0725,  0.0537,
         0.0294,  0.0919,  0.0547,  0.0913,  0.1103,  0.1053,  0.0560,  0.1053,
         0.0598,  0.0470,  0.1068,  0.1043,  0.0632,  0.1070,  0.0955,  0.1063,
         0.1189,  0.0902,  0.0996,  0.1094,  0.0960,  0.0886,  0.1013,  0.1048,
         0.1011,  0.0280,  0.1327,  0.1094,  0.0978,  0.1053,  0.1027,  0.0527,
         0.1123,  0.1003,  0.1071,  0.0280,  0.0967,  0.0919,  0.0575,  0.1056,
         0.1039,  0.1124,  0.0928,  0.09

In [ ]:
criterion = DeviationLoss()
no_decay = ['bias']
# opt_parameter = [
#     {
#         "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
#         "weight_decay": 1e-2
#     },
#     {
#         "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
#         "weight_decay": 0
#     },
# ]
optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=1e-5)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
alpha = 0.01

model.zero_grad()
model.cuda()
model.train()
for epoch in range(epochs):
  tbar = tqdm(dev_dataloader)
  train_loss = 0
  for i, batch in enumerate(tbar):
    x, y = batch["x"].cuda(), batch["y"].cuda()
    score = model(x)
    loss = criterion(score.reshape(1,-1)[0], y)

    l2 = torch.tensor(0., requires_grad=True).cuda()
    for n, p in model.named_parameters():
      if not any(nd in n for nd in no_decay):
        l2 += torch.norm(p)**2
    loss += l2*alpha

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    train_loss += loss.item()
    # scheduler.step()
    tbar.set_description('Epoch:%d, Train loss: %.3f' % (epoch, train_loss / (i + 1)))

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
dev_test_dataset = DevDataset(x_test, mode="test")
dev_test_dataloader = DataLoader(dev_test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

In [ ]:
model.eval()

pred_list = []
for batch in dev_test_dataloader:
  x = batch["x"].cuda()

  with torch.no_grad():
    pred = model(x)
    pred_list.append(pred.cpu().numpy())
pred_list = np.vstack(pred_list).reshape(1, -1)[0]

In [ ]:
i = 0
rauc[i], ap[i] = aucPerformance(pred_list, y_test)

AUC-ROC: 0.8088, AUC-PR: 0.3015


論文では、AUC-ROC:  0.783±0.003、AUC-PR: 0.274±0.011なので、少し精度が良かった

In [ ]:
model

DevNet(
  (devnet): ShortDevNet(
    (linear_0): Linear(in_features=21, out_features=20, bias=True)
    (linear_1): Linear(in_features=20, out_features=1, bias=True)
    (relu): ReLU()
  )
)

In [ ]:
summary(model, torch.zeros((1, 21)).cuda())

                         Kernel Shape Output Shape Params Mult-Adds
Layer                                                              
0_devnet.Linear_linear_0     [21, 20]      [1, 20]  440.0     420.0
1_devnet.ReLU_relu                  -      [1, 20]      -         -
2_devnet.Linear_linear_1      [20, 1]       [1, 1]   21.0      20.0
---------------------------------------------------------------------
                      Totals
Total params           461.0
Trainable params       461.0
Non-trainable params     0.0
Mult-Adds              440.0


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_devnet.Linear_linear_0,"[21, 20]","[1, 20]",440.0,420.0
1_devnet.ReLU_relu,-,"[1, 20]",NaN,NaN
2_devnet.Linear_linear_1,"[20, 1]","[1, 1]",21.0,20.0
